In [2]:
import pandas as pd
import numpy as np
import json
import os
from tabulate import tabulate

In [22]:
cutoff = 16
potential = 'DW'
folder_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\VQD\Destructive SWAP\VaryBeta\{}".format(potential)
betas = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

In [23]:
converged_only = False
all_data = []

for beta in betas:

    median = []
    mean = []
    min = []
    e0 = []
    e1 = []
    e2 = []
    ratios = []

    data_path = folder_path + "\\{}\\{}_{}.json".format(beta,potential, cutoff)
    
    with open(data_path, 'r') as file:
        data = json.load(file)

    num_VQD = data['num_VQD']

    converged = [all(x) for x in data['success']]
    converged_indices = [i for i, success in enumerate(converged) if success]
    converged_runs = sum(converged)

    if converged_only:
        results = [data['results'][i] for i in converged_indices]
    else:
        results = data['results']

    success_i = [i for i, success in enumerate(data['success']) if all(success)]
    successful_results = [data['results'][i] for i in success_i]
    energies =np.sort(successful_results)

    run_times = [data['run_times'][i] for i in converged_indices]
    time_series = np.array([pd.Timedelta(t).total_seconds() for t in run_times])
    mean_vqe_time = np.mean(time_series)
    time_taken = np.mean(mean_vqe_time)

    for e in energies:
        
        e0.append(e[0])
        e1.append(e[1])
        e2.append(e[2])
        r = abs((e[2] - e[1]) / (e[2] - e[0]))
        ratios.append(r)

    e0_exact = data['exact_eigenvalues'][0]
    e1_exact = data['exact_eigenvalues'][1]
    e2_exact = data['exact_eigenvalues'][2]
    exact_ratio = abs((e2_exact - e1_exact) / (e2_exact - e0_exact))

    row = {
    "beta": beta,
    'Converged Runs': f"{converged_runs}/100",
    'Mean VQD time (s)': f"{time_taken:.2f}",
    #"e0_exact": e0_exact,
    #"e1_exact": e1_exact,
    #"e2_exact": e2_exact,
    #"e0": np.median(e0),
    #"e1": np.median(e1),
    #"e2": np.median(e2),
    "E01": e1_exact - e0_exact,
    "E12": e2_exact - e1_exact,
    "exact_ratio": exact_ratio,
    "median_ratio": np.median(ratios),
    "delta_ratio": abs(exact_ratio-np.median(ratios))
    #"mean_ratio": np.mean(ratios),
    #"min_ratio": np.min(ratios),
    }

    all_data.append(row)

In [24]:
df = pd.DataFrame(all_data)
df

,beta,Converged Runs,Mean VQD time (s),E01,E12,exact_ratio,median_ratio,delta_ratio
0,0.5,100/100,189.09,0.000042,1.842481,0.999977,0.993342,0.006636
1,1.0,100/100,209.90,0.000042,1.842481,0.999977,0.993348,0.006629
2,1.5,100/100,195.41,0.000042,1.842481,0.999977,0.992166,0.007812
3,2.0,100/100,210.93,0.000042,1.842481,0.999977,0.991622,0.008356
4,2.5,100/100,225.38,0.000042,1.842481,0.999977,0.989069,0.010909
5,3.0,100/100,266.06,0.000042,1.842481,0.999977,0.986165,0.013812


In [30]:
latex_table = tabulate(df.values, headers=df.columns, tablefmt="latex", floatfmt=".3e")
print(latex_table)

\begin{tabular}{rlrrrrrr}
\hline
      beta & Converged Runs   &   Mean VQD time (s) &       E01 &       E12 &   exact\_ratio &   median\_ratio &   delta\_ratio \\
\hline
 5.000e-01 & 100/100          &           1.891e+02 & 4.158e-05 & 1.842e+00 &     1.000e+00 &      9.933e-01 &     6.636e-03 \\
 1.000e+00 & 100/100          &           2.099e+02 & 4.158e-05 & 1.842e+00 &     1.000e+00 &      9.933e-01 &     6.629e-03 \\
 1.500e+00 & 100/100          &           1.954e+02 & 4.158e-05 & 1.842e+00 &     1.000e+00 &      9.922e-01 &     7.812e-03 \\
 2.000e+00 & 100/100          &           2.109e+02 & 4.158e-05 & 1.842e+00 &     1.000e+00 &      9.916e-01 &     8.356e-03 \\
 2.500e+00 & 100/100          &           2.254e+02 & 4.158e-05 & 1.842e+00 &     1.000e+00 &      9.891e-01 &     1.091e-02 \\
 3.000e+00 & 100/100          &           2.661e+02 & 4.158e-05 & 1.842e+00 &     1.000e+00 &      9.862e-01 &     1.381e-02 \\
\hline
\end{tabular}
